# TCC: 
# Code: 
# Author: Anísio Pereira Batista Filho

In [2]:
##Essentials
import os
import csv
import numpy as np ##Numpy
import pandas as pd ##Pandas
##Utils
import re
import unicodedata
from datetime import datetime
from itertools import islice
from tqdm.auto import tqdm
import time
import timeit

In [3]:
##Eliminar duplicatas em um dataframe:
def df_drop_duplicates(df):
    df = df.drop_duplicates(subset='tweet_id', keep='first')
    return df

In [4]:
##Comparador de treinamentos (compara dois treinamentos e cria um conjunto de iguais e um de diferentes)
def compare_trainings(dfA, dfB, nome):
    listC = []
    listD = []
    for a in dfA.itertuples():
        for b in dfB.itertuples():
            if a.tweet_id == b.tweet_id:
                if a.label == b.label:
                    dictC = dict({'tweet_id':a.tweet_id,'created_at':a.created_at,'user_location':a.user_location,'tweet_text':a.tweet_text,'label_A':a.label,'label_B':b.label,'pct_certainty_A':a.pct_certainty,'pct_certainty_B':b.pct_certainty})
                    listC.append(dictC)
                else:
                    dictD = dict({'tweet_id':a.tweet_id,'created_at':a.created_at,'user_location':a.user_location,'tweet_text':a.tweet_text,'label_A':a.label,'label_B':b.label,'pct_certainty_A':a.pct_certainty,'pct_certainty_B':b.pct_certainty})
                    listD.append(dictD)     

    dfC = pd.DataFrame(listC, columns=['tweet_id','created_at','user_location','tweet_text','label_A','label_B','pct_certainty_A','pct_certainty_B'])
    dfD = pd.DataFrame(listD, columns=['tweet_id','created_at','user_location','tweet_text','label_A','label_B','pct_certainty_A','pct_certainty_B'])

    dfC.to_csv(f"data/corpus_labeled/iguais/{nome}[iguais].csv", sep=",", index=False)
    dfD.to_csv(f"data/corpus_labeled/diferentes/{nome}[diferentes].csv", sep=",", index=False)

In [5]:
##Calculo de Cohen Kappa:
def cohen_kappa(df, dfA, dfB):
    ##Criando dicionário de rótuloxquantidade com as rotulações do avaliador A:
    labelerA = dict()
    fake_news = opinion = news = 0
    for row in dfA.itertuples():
        if row.label == 1:
            fake_news += 1
            labelerA['fake_news'] = fake_news
        else: 
            if row.label == 0:
                opinion += 1
                labelerA['opinion'] = opinion
            else:
                if row.label == -1:
                    news += 1
                    labelerA['news'] = news
    print('Rotulador A: ',labelerA)

    ##Calculando a probabilidade de concordancia randomica do avaliador A:
    peA = dict()
    peA['fake_news'] = labelerA['fake_news']/dfA.shape[0]
    peA['opinion'] = labelerA['opinion']/dfA.shape[0]
    peA['news'] = labelerA['news']/dfA.shape[0]
    print('peA: ',peA)

    ##Criando dicionário de rótuloxquantidade com as rotulações do avaliador B:
    labelerB = dict()
    fake_news = opinion = news = 0
    for row in dfB.itertuples():
        if row.label == 1:
            fake_news += 1
            labelerB['fake_news'] = fake_news
        else: 
            if row.label == 0:
                opinion += 1
                labelerB['opinion'] = opinion
            else:
                if row.label == -1:
                    news += 1
                    labelerB['news'] = news
    print('Rotulador B: ',labelerB)

    ##Calculando a probabilidade de concordancia randomica do avaliador B:
    peB = dict()
    peB['fake_news'] = labelerB['fake_news']/dfB.shape[0]
    peB['opinion'] = labelerB['opinion']/dfB.shape[0]
    peB['news'] = labelerB['news']/dfB.shape[0]
    print('peB: ',peB)

    ##Calculando a probabilidade de ambos os avaliadores (para fake_news, opinion e news):
    pe = dict()
    pe['fake_news'] = peA['fake_news']*peB['fake_news']
    pe['opinion'] = peA['opinion']*peB['opinion']
    pe['news'] = peA['news']*peB['news']
    print('pe:',pe)

    ##Calculando a probabilidade de aceitação total:
    Pr = pe['fake_news']+pe['opinion']+pe['news']
    print('Pr: ',Pr)

    ##Calculando Po
    Po = df.shape[0]/dfA.shape[0]
    print('Po: ',Po)
    ##Calculando Kappa:
    K = (Po - Pr)/(1 - Pr)
    print('K: ',K)

## Função main:

### Rotulação 1

In [10]:
###Abertura de arquivo e criação do dataframe:
df1_file = pd.read_csv("data/original_corpus_to_label/base_para_rotulaçao1.csv", sep=",")
df1 = pd.DataFrame(df1_file)
df1 = df1.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", df1.shape[0])
df1.sample(5)

Tamanho: 1963


tweet_id                      created_at       user_location  \
1162  1349414463301160969  Wed Jan 13 17:54:13 +0000 2021     Gado Bravo - PB   
1252  1348056217294426117  Sat Jan 09 23:57:02 +0000 2021                 NaN   
1424  1348046934745952257  Sat Jan 09 23:20:09 +0000 2021                 NaN   
918   1346566035915354113  Tue Jan 05 21:15:35 +0000 2021  Santa Fe Argentina   
1423  1347316984527118339  Thu Jan 07 22:59:35 +0000 2021         Planet Blue   

                                             tweet_text  
1162  @diegodrummer10 @psmoreno @rosa_riscala kkkkkk...  
1252  @CNNBrasil O problema é que não temos vacinas ...  
1424  @hiddletstark ele bota TUDO na boca amg não da...  
918   @ellitoral Que pena che. Están como Pfizer eso...  
1423  @senadorcidgomes Sai daí mal elemento ##COVID1...

In [11]:
###Abertura de arquivo e criação do dataframe:
dfA1_file = pd.read_csv("data/corpus_labeled/rotulaçao1[anisiofilho].csv", sep=",")
dfA1 = pd.DataFrame(dfA1_file)
dfA1 = dfA1.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfA1.shape[0])
dfA1.sample(5)

Tamanho: 1963


tweet_id                      created_at  \
794   1348411158936616961  Sun Jan 10 23:27:27 +0000 2021   
1330  1349142020279984128  Tue Jan 12 23:51:38 +0000 2021   
1992  1347235811624955907  Thu Jan 07 17:37:02 +0000 2021   
611   1348054011866476545  Sat Jan 09 23:48:16 +0000 2021   
1141  1346588934881599488  Tue Jan 05 22:46:35 +0000 2021   

                user_location  \
794                       NaN   
1330  RIO DE JANEIRO - BRASIL   
1992                      NaN   
611                    Mexico   
1141      Esperantina, Brasil   

                                             tweet_text  label  pct_certainty  
794                           @barceIox asma ou corona?      0            100  
1330  @GloboNews "Sacrificando a eficácia". Ouvi iss...      1             75  
1992  @UOL URGENTE - VACINA DE DORIA TEM A MENOR EFI...     -1            100  
611   ...violência baseada em gênero e para prevençã...      0             25  
1141  A vacina da @pfizer e a Moderna não tem condiç...     -1             25

In [12]:
###Verificação da quantidade de tuítes por rótulo
dfA1.groupby('label').size()

label
-1    606
 0    827
 1    530
dtype: int64

In [13]:
###Abertura de arquivo e criação do dataframe:
dfB1_file = pd.read_csv("data/corpus_labeled/rotulaçao1[debora].csv", sep=",")
dfB1 = pd.DataFrame(dfB1_file)
dfB1 = dfB1.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfB1.shape[0])
dfB1.sample(5)

Tamanho: 1963


tweet_id                      created_at  \
1867  1346960841648861190  Wed Jan 06 23:24:24 +0000 2021   
1474  1348054212215779328  Sat Jan 09 23:49:04 +0000 2021   
965   1346605921481469954  Tue Jan 05 23:54:05 +0000 2021   
1908  1347639140423241733  Fri Jan 08 20:19:43 +0000 2021   
561   1349093149856190464  Tue Jan 12 20:37:26 +0000 2021   

             user_location                                         tweet_text  \
1867    São Paulo - Brasil  @brodithiago O Jd Ibirapuera está localizado n...   
1474  Porto Alegre, Brasil  @parolin_ricardo "Nós, pesquisadores, argument...   
965        No seu tempo...          @hey_mika A estreia da vacina do covid 19   
1908        São Paulo — SP  @gui_pinheiro A Oxford/AstraZeneca pediu no Re...   
561                    NaN  @mellziland @analise_covid19 @obscovid19br @UP...   

      label  pct_certainty  
1867      0            100  
1474     -1            100  
965       0            100  
1908     -1             75  
561       0            100

In [14]:
###Verificação da quantidade de tuítes por rótulo
dfB1.groupby('label').size()

label
-1     414
 0    1067
 1     482
dtype: int64

### Chamadas de funções:

In [37]:
compare_trainings(dfA1,dfB1,'rotulaçao1')

In [16]:
###Abertura de arquivo e criação do dataframe:
dfC1_file = pd.read_csv("data/corpus_labeled/iguais/rotulaçao1[iguais].csv", sep=",")
dfC1 = pd.DataFrame(dfC1_file)
dfC1 = dfC1.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfC1.shape[0])
dfC1.sample(5)

Tamanho: 1300


tweet_id                      created_at  \
858   1346913599932018696  Wed Jan 06 20:16:41 +0000 2021   
1106  1348053235123302405  Sat Jan 09 23:45:11 +0000 2021   
298   1348769864610160640  Mon Jan 11 23:12:49 +0000 2021   
761   1349141549171531779  Tue Jan 12 23:49:45 +0000 2021   
380   1347694364013813762  Fri Jan 08 23:59:09 +0000 2021   

                       user_location  \
858                              NaN   
1106                  No seu coração   
298                           Newark   
761           Rio de Janeiro, Brasil   
380   bieber has noted 19/10/17 13:2   

                                             tweet_text  label_A  label_B  \
858   @marco_olvra A vacina da Sinovac não foi aprov...        1        1   
1106  @miojovegano Num paguei nem a minha Corona dei...        0        0   
298   @mab_sp125 @claudioedantas Existe alguma coisa...        0        0   
761   @mr_casella @diogomainardi A Vacina de Oxford,...       -1       -1   
380   @ciciliacampos_ minha mae tava explicando isso...        1        1   

      pct_certainty_A  pct_certainty_B  
858                25              100  
1106              100              100  
298                75               75  
761                25               25  
380                25               75

In [17]:
dfC1.groupby('label_A').size()
#dfC1.groupby('label_B').size()

label_A
-1    320
 0    705
 1    275
dtype: int64

In [18]:
###Abertura de arquivo e criação do dataframe:
dfD1_file = pd.read_csv("data/corpus_labeled/diferentes/rotulaçao1[diferentes].csv", sep=",")
dfD1 = pd.DataFrame(dfD1_file)
dfD1 = dfD1.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfD1.shape[0])
dfD1.sample(5)

Tamanho: 663


tweet_id                      created_at  \
135  1348770301128138754  Mon Jan 11 23:14:33 +0000 2021   
497  1347533589769027584  Fri Jan 08 13:20:18 +0000 2021   
165  1347566355390885890  Fri Jan 08 15:30:30 +0000 2021   
99   1347323095158321153  Thu Jan 07 23:23:52 +0000 2021   
323  1348771326287368199  Mon Jan 11 23:18:37 +0000 2021   

                user_location  \
135  Third Rock, Solar System   
497             Sinop, Brasil   
165              Niterói - RJ   
99          São Paulo, Brazil   
323                       NaN   

                                            tweet_text  label_A  label_B  \
135  @anvisa_oficial e você podem acompanhar aqui q...        0        1   
497            @sputnik_brasil Ae acaba o whatsApp!!!!        1        0   
165  @affalva Tem 3 erros aí.  Por motivos diferent...       -1        1   
99   @thaisplicandoo @alvarodias_ Thais, vamos faze...        1        0   
323  @BlogdoNoblat Pior é ele querer inventar e dar...        1       -1   

     pct_certainty_A  pct_certainty_B  
135               75               50  
497               50              100  
165               25              100  
99                25               50  
323               25              100

In [19]:
dfD1.groupby('label_A').size()

label_A
-1    286
 0    122
 1    255
dtype: int64

In [20]:
dfD1.groupby('label_B').size()

label_B
-1     94
 0    362
 1    207
dtype: int64

In [21]:
cohen_kappa(dfC1, dfA1, dfB1)

Rotulador A:  {'news': 606, 'opinion': 827, 'fake_news': 530}
peA:  {'fake_news': 0.26999490575649515, 'opinion': 0.4212939378502292, 'news': 0.3087111563932756}
Rotulador B:  {'opinion': 1067, 'news': 414, 'fake_news': 482}
peB:  {'fake_news': 0.24554253693326542, 'opinion': 0.543555781966378, 'news': 0.2109016811003566}
pe: {'fake_news': 0.06629523411850773, 'opinion': 0.228996755825876, 'news': 0.06510770185777692}
Pr:  0.36039969180216064
Po:  0.6622516556291391
K:  0.4719384277307297


### Rotulação 2

In [51]:
###Abertura de arquivo e criação do dataframe:
df2_file = pd.read_csv("data/original_corpus_to_label/base_para_rotulaçao2.csv", sep=",")
df2 = pd.DataFrame(df2_file)
df2 = df.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", df2.shape[0])
df2.sample(5)

Tamanho: 3037


tweet_id                      created_at  \
1438  1348054946978156544  Sat Jan 09 23:51:59 +0000 2021   
865   1347331845839597569  Thu Jan 07 23:58:38 +0000 2021   
1037  1348772316860313602  Mon Jan 11 23:22:33 +0000 2021   
2599  1348074739294740482  Sun Jan 10 01:10:38 +0000 2021   
2544  1348050234677616649  Sat Jan 09 23:33:16 +0000 2021   

               user_location  \
1438                     NaN   
865                      NaN   
1037          Belo Horizonte   
2599                     NaN   
2544  Rio de Janeiro, Brasil   

                                             tweet_text  
1438  @thaissnnt @jogueinovitoria Isso já é abuso e ...  
865   @sami_oliveer Garganta ruim? Não me passou cor...  
1037  @thordebatom sim, mano, a gente tava zoando su...  
2599  Mulher tem ataque epiléptico após #Vacina do C...  
2544  @KropfSimone @Gilbert70056731 @marivarella Aqu...

In [54]:
###Abertura de arquivo e criação do dataframe:
dfA2_file = pd.read_csv("data/corpus_labeled/rotulaçao2[anisiofilho].csv", sep=",")
dfA2 = pd.DataFrame(dfA2_file)
dfA2 = dfA2.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfA2.shape[0])
dfA2.sample(5)

Tamanho: 3037


tweet_id                      created_at  \
2815  1347602442016448517  Fri Jan 08 17:53:54 +0000 2021   
561   1348406957825265665  Sun Jan 10 23:10:45 +0000 2021   
777   1348417494105661442  Sun Jan 10 23:52:37 +0000 2021   
2202  1349505734912978952  Wed Jan 13 23:56:54 +0000 2021   
2879  1349139516376969217  Tue Jan 12 23:41:41 +0000 2021   

              user_location  \
2815      São Paulo, Brasil   
561                     NaN   
777                     NaN   
2202                    NaN   
2879  Volta Redonda, Brasil   

                                             tweet_text  label  pct_certainty  \
2815  @g1 O gov do RJ informou nesta 6a-feira, em co...     -1             75   
561   @Cecillia É diário esse ódio: - Mortos por Cov...      0            100   
777   @simone_ss2019 @onyxlorenzoni Aonde está este ...      0            100   
2202  Pinga com alho  55,50%  Wiskey paraguaio  55,4...      1             50   
2879  Tomar um gelo na calçada em Campo Grande. Que ...      0            100   

      doubt  
2815    0.0  
561     0.0  
777     0.0  
2202    0.0  
2879    0.0

In [55]:
dfA2.groupby('label').size()

label
-1     743
 0    1891
 1     403
dtype: int64

In [56]:
###Abertura de arquivo e criação do dataframe:
dfB2_file = pd.read_csv("data/corpus_labeled/rotulaçao2[andre].csv", sep=",")
dfB2 = pd.DataFrame(dfB2_file)
dfB2 = dfB2.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfB2.shape[0])
dfB2.sample(5)

Tamanho: 3037


tweet_id                      created_at  user_location  \
673   1347583401105227776  Fri Jan 08 16:38:14 +0000 2021  Belém, Brasil   
2853  1348052010818543621  Sat Jan 09 23:40:19 +0000 2021            NaN   
92    1347693921208590336  Fri Jan 08 23:57:24 +0000 2021            NaN   
2623  1348764089070342150  Mon Jan 11 22:49:52 +0000 2021            NaN   
2663  1348737733414576130  Mon Jan 11 21:05:08 +0000 2021            NaN   

                                             tweet_text  label  pct_certainty  \
673               @Sputnik_kintupS Tem baby  Parece né?    0.0          100.0   
2853  @derflecha Bolsonaro não quer que Dória seja o...    1.0          100.0   
92    @CalledMovies Obrigado! Vacina pra todos! Vamo...    0.0          100.0   
2623  @EduardoBraga_AM Boa noite, peço sua ajuda aqu...    0.0          100.0   
2663  @LainoPletsch @o_antagonista Os EUA país sede ...    0.0          100.0   

      doubt  
673     0.0  
2853    0.0  
92      0.0  
2623    0.0  
2663    0.0

In [57]:
dfB2.groupby('label').size()

label
-1.0     485
 0.0    2103
 1.0     444
dtype: int64

### Chamadas de funções:

In [58]:
compare_trainings(dfA2,dfB2,'rotulaçao2')

In [59]:
dfC2_file = pd.read_csv("data/corpus_labeled/iguais/rotulaçao2[iguais].csv", sep=",")
dfC2 = pd.DataFrame(dfC2_file)
dfC2 = dfC2.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfC2.shape[0])
dfC2.sample(5)

Tamanho: 2300


tweet_id                      created_at  \
2232  1347331768442114049  Thu Jan 07 23:58:20 +0000 2021   
1207  1347387532816871425  Fri Jan 08 03:39:55 +0000 2021   
1802  1347694099885928458  Fri Jan 08 23:58:07 +0000 2021   
609   1348776782808440837  Mon Jan 11 23:40:18 +0000 2021   
2207  1346519888127545344  Tue Jan 05 18:12:13 +0000 2021   

                   user_location  \
2232  Brasília, Distrito Federal   
1207                         NaN   
1802                         NaN   
609           Buenos Aires, ARG.   
2207                         NaN   

                                             tweet_text  label_A  label_B  \
2232   @viirgulas vai tomar a vacina e voltar né safado        0      0.0   
1207  @camilalcorsini Não vai ter porra nenhuma de C...        1      1.0   
1802  @juaotenorio @rmattohs é né fia, não tem vacin...        0      0.0   
609   @SenorTesteo @Palapo2020 @CNNEE Brasil compró ...       -1     -1.0   
2207  @OGabiiel @LCSG10 @MarcosRDuarte @rafaelglauss...        0      0.0   

      pct_certainty_A  pct_certainty_B  
2232              100            100.0  
1207               74            100.0  
1802              100            100.0  
609                25            100.0  
2207              100            100.0

In [61]:
dfC2.groupby('label_A').size()
#dfC.groupby('label_B').size()

label_A
-1     394
 0    1694
 1     212
dtype: int64

In [62]:
dfD2_file = pd.read_csv("data/corpus_labeled/diferentes/rotulaçao2[diferentes].csv", sep=",")
dfD2 = pd.DataFrame(dfD2_file)
dfD2 = dfD2.drop_duplicates(subset='tweet_id', keep='first')
print("Tamanho:", dfD2.shape[0])
dfD2.sample(5)

Tamanho: 737


tweet_id                      created_at user_location  \
244  1346541429661851652  Tue Jan 05 19:37:49 +0000 2021           NaN   
211  1348354330022326274  Sun Jan 10 19:41:38 +0000 2021           NaN   
184  1349139606747418624  Tue Jan 12 23:42:02 +0000 2021           NaN   
110  1348774575090372610  Mon Jan 11 23:31:32 +0000 2021           NaN   
517  1346966872303034368  Wed Jan 06 23:48:22 +0000 2021           NaN   

                                            tweet_text  label_A  label_B  \
244  @Livroscons @Biakicis @jairbolsonaro Sobre OXF...       -1      0.0   
211  @msffeliz Não. É que Anvisa quer liberar a Oxf...        0      1.0   
184  @AlexRondas @sccp_zi @CNNBrasil ”O material pa...        0      1.0   
110  @thiagosgbr @luizacaires3 A Pfizer, por exempl...       -1      0.0   
517  @apyus Bozo,vc é incompetente e tá levando cen...        0      1.0   

     pct_certainty_A  pct_certainty_B  
244               75            100.0  
211              100            100.0  
184              100             50.0  
110               50             50.0  
517               25             50.0

In [63]:
dfD2.groupby('label_A').size()

label_A
-1    349
 0    197
 1    191
dtype: int64

In [64]:
dfD2.groupby('label_B').size()

label_B
-1.0     91
 0.0    409
 1.0    232
dtype: int64

In [65]:
cohen_kappa(dfC2, dfA2, dfB2)

Rotulador A:  {'opinion': 1891, 'fake_news': 403, 'news': 743}
peA:  {'fake_news': 0.13269674020414884, 'opinion': 0.6226539348040829, 'news': 0.2446493249917682}
Rotulador B:  {'fake_news': 444, 'news': 485, 'opinion': 2103}
peB:  {'fake_news': 0.14619690484030293, 'opinion': 0.6924596641422457, 'news': 0.15969706947645704}
pe: {'fake_news': 0.019399852700244347, 'opinion': 0.431162734571283, 'news': 0.03906978025057872}
Pr:  0.4896323675221061
Po:  0.7573263088574251
K:  0.5245119876345489


## Unindo dataframes

In [71]:
###Rotulação iguais
frames_iguais = [dfC1, dfC2]

result_iguais = pd.concat(frames_iguais)
print("Tamanho:", result_iguais.shape[0])
result_iguais.sample(5)

Tamanho: 3600


tweet_id                      created_at      user_location  \
799   1347627097116835845  Fri Jan 08 19:31:52 +0000 2021                NaN   
1669  1348407257369817088  Sun Jan 10 23:11:57 +0000 2021             Brasil   
1989  1349502740423176192  Wed Jan 13 23:45:00 +0000 2021             Brasil   
1145  1347640010502246409  Fri Jan 08 20:23:11 +0000 2021                NaN   
98    1346606222477287425  Tue Jan 05 23:55:16 +0000 2021  São Paulo, Brasil   

                                             tweet_text  label_A  label_B  \
799   @xicograziano não existe vacina desenvolvida p...        1      1.0   
1669  @jcarlos_46 Antes do corona tinha ,agora n sei rs        0      0.0   
1989  @LYDIALAGRUTTA @fogaca_henrique @masterchefbr ...        0      0.0   
1145  @laurojardim De repente, você põe a mão por de...        0      0.0   
98    @guilherme_amado Eu também podia estar passean...        0      0.0   

      pct_certainty_A  pct_certainty_B  
799                50            100.0  
1669              100            100.0  
1989              100            100.0  
1145              100            100.0  
98                100             50.0

In [72]:
result_iguais.to_csv(f"data/corpus_labeled/iguais/rotulacao_uniao[iguais].csv", sep=",", index=False)

In [73]:
###Rotulação diferentes
frames_diferentes = [dfD1, dfD2]

result_diferentes = pd.concat(frames_diferentes)
print("Tamanho:", result_diferentes.shape[0])
result_diferentes.sample(5)

Tamanho: 1400


tweet_id                      created_at  \
373  1348760474817597441  Mon Jan 11 22:35:30 +0000 2021   
576  1348353416775544833  Sun Jan 10 19:38:00 +0000 2021   
360  1347598488444547075  Fri Jan 08 17:38:11 +0000 2021   
585  1348336992673816577  Sun Jan 10 18:32:44 +0000 2021   
246  1349043568028090368  Tue Jan 12 17:20:25 +0000 2021   

              user_location  \
373       São Paulo, Brazil   
576                     NaN   
360  Belo Horizonte, Brasil   
585                     NaN   
246                     NaN   

                                            tweet_text  label_A  label_B  \
373  @Rconstantino há pouco vc comparou #covid19 co...       -1      0.0   
576  @amadeu_penzin @octavio_guedes Os resultados d...       -1      1.0   
360  @NoSense_BR Vi ontem numa reportagem que a vac...       -1      1.0   
585  @acarolthing @PaulaFaoth @mari_fonseca Sim, el...       -1      0.0   
246  @taoquei1 É a vacina que temos, já que o desgo...       -1      0.0   

     pct_certainty_A  pct_certainty_B  
373               50            100.0  
576              100            100.0  
360               75            100.0  
585               75            100.0  
246               25            100.0

In [74]:
result_diferentes.to_csv(f"data/corpus_labeled/diferentes/rotulacao_uniao[diferentes].csv", sep=",", index=False)

In [7]:
df_completo_iguais = pd.read_csv("data/corpus_labeled/iguais/rotulacao_uniao[iguais].csv", sep=",")

In [8]:
df_completo_iguais.groupby('label_A').size()

label_A
-1     714
 0    2399
 1     487
dtype: int64

In [9]:
df_completo_iguais.groupby('label_B').size()

label_B
-1.0     714
 0.0    2399
 1.0     487
dtype: int64

### Miscelanea

In [46]:
#
if dfA1.tweet_id is not dfB1.tweet_id:
    print(dfA1.tweet_id)

0       1348780962394603522
1       1346534190335074310
2       1348777478882521097
3       1349140630753210373
4       1348749014964056065
               ...         
1995    1347631155420475392
1996    1346428376731570176
1997    1347993007132463109
1998    1347198002788757504
1999    1349499526718124034
Name: tweet_id, Length: 1963, dtype: object


In [47]:
dfA1 = dfA1.assign(InDf2=dfA1.tweet_id.isin(dfB1.tweet_id).astype(int))

In [48]:
dfA1.assign(InDf2=dfA1.tweet_id.isin(dfB1.tweet_id).astype(int))

tweet_id                      created_at  \
0     1348780962394603522  Mon Jan 11 23:56:55 +0000 2021   
1     1346534190335074310  Tue Jan 05 19:09:03 +0000 2021   
2     1348777478882521097  Mon Jan 11 23:43:04 +0000 2021   
3     1349140630753210373  Tue Jan 12 23:46:06 +0000 2021   
4     1348749014964056065  Mon Jan 11 21:49:58 +0000 2021   
...                   ...                             ...   
1995  1347631155420475392  Fri Jan 08 19:47:59 +0000 2021   
1996  1346428376731570176  Tue Jan 05 12:08:35 +0000 2021   
1997  1347993007132463109  Sat Jan 09 19:45:52 +0000 2021   
1998  1347198002788757504  Thu Jan 07 15:06:48 +0000 2021   
1999  1349499526718124034  Wed Jan 13 23:32:14 +0000 2021   

                       user_location  \
0                   honeymoon Avenue   
1                  São Paulo, Brasil   
2                                NaN   
3                             Brasil   
4                          Barcelona   
...                              ...   
1995                             NaN   
1996                          Israel   
1997                       São Paulo   
1998  Beijing, People's Republic of    
1999                             NaN   

                                             tweet_text  label  pct_certainty  \
0     Fora brigas de trânsito que sim amados causam ...     -1             25   
1     @HugoMachadoFil3 @quadrelli_marco A parceria c...     -1             75   
2     @jrmestrado @MarceloAdnet Estamos numa PANDEMI...      0             50   
3     @danielgirardi83 @Fenovais28 @mab_sp125 @samyd...      0             75   
4       @edisonrd1977 Es peligrosa la vacuna de Pfizer?      0            100   
...                                                 ...    ...            ...   
1995  @sputnik_brasil @BLOGdoPEPE Foda-se o monte de...      0            100   
1996  Ainda em 2020, o Ministério da Saúde comprou d...      1             75   
1997  @coisiquinha @Cambricoli @ekallas @leosbastos ...      0             50   
1998  Bm explicar: uma eficácia menor que os compost...     -1            100   
1999  @CNNBrasil Já que seremos praticamente obrigad...      0             75   

      InDf2  
0         1  
1         1  
2         1  
3         1  
4         1  
...     ...  
1995      1  
1996      1  
1997      1  
1998      1  
1999      1  

[1963 rows x 7 columns]

In [49]:
dfA1.groupby('InDf2').size()

InDf2
1    1963
dtype: int64

In [50]:
dfA1.loc[dfA1['InDf2'] == 0]

Empty DataFrame
Columns: [tweet_id, created_at, user_location, tweet_text, label, pct_certainty, InDf2]
Index: []